In [28]:
from datasets import load_dataset
from tqdm import tqdm
import torch
   
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
# dataset = load_dataset('cifar10', cache_dir='/data')
# dataset.set_format(type='torch')
# dataset


In [30]:
# dataloader = torch.utils.data.DataLoader(dataset['train'], batch_size=8)

In [31]:
# dataset['train'].

In [59]:
from torchvision import datasets 
from torchvision import transforms
#transform = transforms.Compose([transforms.Resize((224,224)),  transforms.ToTensor()])
transform = transforms.Compose([  transforms.ToTensor(), transforms.Lambda(lambda x: x.to(device))])
to_device_transform = transforms.Compose([
    
    transforms.Lambda(lambda x: x.to(device))])

dataset = datasets.CIFAR10(root='data', train = True, download=True, transform = transform)#, target_transform=to_device_transform)
#dataloader = data_loader(data_dir='./data', batch_size=32)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


In [60]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride = stride, padding = 1),  #outputs the same size on default
            nn.BatchNorm2d(out_channels),
            nn.ReLU()           
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1 ),  #outputs the same size
        nn.BatchNorm2d(out_channels)
        ) 
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if (self.downsample):
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [89]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride = 2, padding = 3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
        
        self.last_conv = torch.nn.Conv2d(in_channels = 512, out_channels = num_classes, kernel_size = 1)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        #print('before avgpool shape is', x.shape)
        
        #x = self.avgpool(x)
        #
        
        
        #x = self.fc(x)
        x = self.last_conv(x)
        
        x = x.view(x.size(0), -1)
        return x
        
    
    
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride), 
                                        nn.BatchNorm2d(planes)
                                    )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
            
        return nn.Sequential(*layers)
    

In [90]:
num_classes = 10
num_epochs = 20
batch_size = 16
learining_rate = 0.001
model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learining_rate, weight_decay = 0.001, momentum = 0.9)

total_step = len(dataset)

In [94]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(tqdm(dataloader)):
        
        
        output = model(images)
        labels = torch.tensor(labels).to(device)

        
        #print(labels.shape)
        #print(output.shape)
        loss = criterion(output, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        

  0%|                                                                                         | 0/1563 [00:00<?, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_12508\795189681.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)
100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:41<00:00, 37.64it/s]


In [93]:
datasets.CIFAR10?

In [95]:
transform = transforms.Compose([  transforms.ToTensor(), transforms.Lambda(lambda x: x.to(device))])
to_device_transform = transforms.Compose([
    
    transforms.Lambda(lambda x: x.to(device))])

dataset = datasets.CIFAR10(root='data', train = False, download=True, transform = transform)#, target_transform=to_device_transform)
#dataloader = data_loader(data_dir='./data', batch_size=32)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


In [96]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))  

Accuracy of the network on the 10000 test images: 73.43 %
